In [13]:
import notebook_common
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    w_word_raw_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
)

p("Import common module")

Import common module


In [4]:
df = pd.read_csv(
    "d:/tmp/transformed_2400.csv",
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
# index_col=["headword", "reading"],
)
df







,NO,书写,P,位置,词性,细分01,词性2,细分02,词性3,细分03,其他,中文意思
0,1,ああ,ああ,NaN,感叹词,NaN,NaN,NaN,NaN,NaN,NaN,啊、嗯、啊呀、唉
1,2,愛,あい,五十音,名词,NaN,动词,动3,NaN,NaN,他,爱情、爱好
2,3,あいさつ,あいさつ,NaN,名词,NaN,动词,动3,NaN,NaN,自,寒暄、致辞
3,4,合図,あいず,NaN,名词,NaN,NaN,NaN,NaN,NaN,NaN,信号
4,5,アイスクリーム,あいすくりーむ,NaN,名词,NaN,NaN,NaN,NaN,NaN,NaN,冰激凌、雪糕
...,...,...,...,...,...,...,...,...,...,...,...,...
2498,NaN,ユーロ,ゆーろ,NaN,名词,NaN,量词,NaN,NaN,NaN,NaN,欧元
2499,NaN,人参,にんじん,NaN,名词,NaN,NaN,NaN,NaN,NaN,NaN,胡萝卜
2500,NaN,明日,あした,NaN,名词,NaN,副词,NaN,NaN,NaN,NaN,明天、在明天
2501,NaN,今日,きょう,NaN,名词,NaN,副词,NaN,NaN,NaN,NaN,今天、在今天


In [16]:
df = pd.read_csv(
    "d:/tmp/aaa.csv", encoding="utf-8-sig", on_bad_lines="skip", engine="python"
)
pattern = r"[。()（）、､【】％！%→？…·・“”」～「｢｣『』①――＝×（，＿:：,．\[\]\.\?0-9０-９a-zA-Z\uFF21-\uFF3A\uFF41-\uFF5A]"

# 使用 ~ 取反，保留不包含这些字符的行
filtered_df = df[~df["spell"].str.contains(pattern, na=False)]


df_base = pd.read_csv(
    w_word_raw_merged_freq_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
)


df_t = pd.merge(
    df_base,
    filtered_df,
    left_on="headword",
    right_on="spell",
    how="outer",
    indicator="check",
)


to_csv_sig(df_t, "d:/tmp/hhh_del.csv")
# filtered_df

In [3]:
merged_list = ["a|在佣人", "b|在有人", "c|我貌有"]
df = pd.DataFrame(merged_list, columns=["final_str"])
split_columns = df['final_str'].str.split('|', expand=True)
df['Key'] = split_columns[0] # 分割后的第一列 (索引为 0)
df['Value'] = split_columns[1] # 分割后的第二列 (索引为 1)
df

,final_str,Key,Value
0,a|在佣人,a,在佣人
1,b|在有人,b,在有人
2,c|我貌有,c,我貌有
